# Multi-Criteria Decision Analysis (MCDA)

In [18]:
# Import necessary libraries
import numpy as np
import os
import pandas as pd

from math import isclose
from scipy.stats import zscore, spearmanr
from SALib.sample import saltelli
from SALib.analyze import sobol

In [19]:
# Constants
CSV_PATH = 'olympic.csv'

In [20]:
# D-CRITIC modified
def _double_center(D):
    row_mean = D.mean(axis=1, keepdims=True)
    col_mean = D.mean(axis=0, keepdims=True)
    grand_mean = D.mean()

    return D - row_mean - col_mean + grand_mean


def distance_covariance(x, y):
    x = np.asarray(x, dtype=float).reshape(-1, 1)
    y = np.asarray(y, dtype=float).reshape(-1, 1)
    Dx = np.abs(x - x.T)
    Dy = np.abs(y - y.T)
    A = _double_center(Dx)
    B = _double_center(Dy)
    n = x.shape[0]
    dcov2 = (A * B).sum() / (n * n)

    return np.sqrt(max(dcov2, 0.0))


def distance_correlation(x, y):
    dcov_xy = distance_covariance(x, y)
    dcov_xx = distance_covariance(x, x)
    dcov_yy = distance_covariance(y, y)
    denom = np.sqrt(dcov_xx * dcov_yy)

    if denom == 0:
        return 0.0
    
    return dcov_xy / denom


def d_critic_weights(df_sub):
    df = df_sub.copy()
    df = df.apply(pd.to_numeric, errors='coerce')
    df = df.loc[:, df.notna().any(axis=0)]
    n_cols = df.shape[1]

    if n_cols == 0:
        raise ValueError("No valid numeric subfeatures provided for this pillar.")

    criteria = df.columns.tolist()
    s = df.std(ddof=1)

    dcor_mat = pd.DataFrame(np.eye(n_cols), index=criteria, columns=criteria)

    for i, ci in enumerate(criteria):
        for j in range(i + 1, n_cols):
            cj = criteria[j]

            try:
                dcor = distance_correlation(df[ci].dropna().values, df[cj].dropna().values)
            except Exception:
                dcor = 0.0

            dcor_mat.loc[ci, cj] = dcor
            dcor_mat.loc[cj, ci] = dcor

    info_content = s * (1 - dcor_mat).sum(axis=1)
    total_info = info_content.sum()

    if isclose(total_info, 0.0):
        raise ValueError("Total information content is zero; check for constant or perfectly correlated columns.")
    
    weights = (info_content / total_info).rename('D-CRITIC_weight')
    
    return weights, dcor_mat


In [21]:
# TOPSIS (pillar level)
def weighted_topsis(df, criteria, weights, impacts):
    X = df[criteria].astype(float).to_numpy()
    weights = np.array(weights, dtype=float)
    weights = weights / weights.sum()
    norm = np.linalg.norm(X, axis=0)
    norm[norm == 0] = 1.0
    R = X / norm
    V = R * weights
    ideal_best = []
    ideal_worst = []

    for i, impact in enumerate(impacts):
        if impact == '+':
            ideal_best.append(V[:, i].max())
            ideal_worst.append(V[:, i].min())
        else:
            ideal_best.append(V[:, i].min())
            ideal_worst.append(V[:, i].max())

    ideal_best = np.array(ideal_best)
    ideal_worst = np.array(ideal_worst)
    dist_best = np.linalg.norm(V - ideal_best, axis=1)
    dist_worst = np.linalg.norm(V - ideal_worst, axis=1)
    score = dist_worst / (dist_best + dist_worst)
    out = df.copy()
    out['topsis_score'] = score
    out['rank'] = out['topsis_score'].rank(ascending=False).astype(int)
    
    return out.sort_values('rank')


In [22]:
# Load csv data
if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"{CSV_PATH} not found. Please place olympic.csv in the notebook directory.")

raw = pd.read_csv(CSV_PATH)
raw.head(10)

,Country,CO2,Country.1,PM,Public Transport,Landfill,Recycle,Water Reuse,Water Stress,Accommodation,Temp_x,Energy Use,Temp_y
0,Afghanistan,110.89522,NaN,62.48616,54.930000,0.162411,NaN,943.80,3.37,55.447607,14.48,5.914411,14.48
1,Africa,265.81262,4.478591,NaN,NaN,NaN,NaN,NaN,NaN,602.819660,NaN,243.772170,NaN
2,Albania,590.60394,17.682753,16.27997,NaN,0.381000,NaN,588.50,3.47,240.485436,13.31,28.292404,13.31
3,Algeria,992.36127,13.853037,22.68313,40.566666,0.304850,8.0,178.90,3.87,1898.525800,24.11,2096.206500,24.11
4,Angola,232.91690,3.928719,27.16477,10.700000,0.167900,NaN,39.76,1.13,461.624420,21.63,736.915400,21.63
5,Antigua and Barbuda,2800.21530,NaN,8.29693,NaN,0.316036,NaN,NaN,NaN,4415.724300,NaN,0.000000,NaN
6,Argentina,1122.05140,22.683275,12.04087,57.547825,0.416704,6.0,NaN,1.83,1987.539440,15.07,693.051150,15.07
7,Armenia,808.78064,13.884646,34.12674,NaN,0.169567,NaN,568.80,2.99,1228.791200,8.64,496.376100,8.64
8,Asia,546.26630,10.079368,NaN,NaN,NaN,NaN,NaN,NaN,2633.228500,NaN,345.818800,NaN
9,Australia,3404.07030,66.409710,8.93323,83.489480,0.560966,42.1,1112.00,2.91,8382.508700,21.96,2293.806600,21.96


In [23]:
# Define pillars and subfeatures
pillars = {
    'transportation': ['Public Transport', 'CO2', 'Country.1'],
    'air_quality': ['PM'],
    'waste': ['Landfill', 'Recycle'],
    'water': ['Water Reuse', 'Water Stress'],
    'accommodation': ['Accommodation', 'Temp_x', 'Temp_y'],
    'energy': ['Energy Use']
}

# Define status map
status = {
    'Public Transport': 'ad',
    'CO2': 'dis',
    'Country.1': 'ad',
    'PM': 'dis',
    'Landfill': 'dis',
    'Recycle': 'ad',
    'Water Reuse': 'ad',
    'Water Stress': 'dis',
    'Accommodation': 'ad',
    'Temp_x': 'ad',
    'Temp_y': 'ad',
    'Energy Use': 'dis'
}

In [24]:
# Columns validation
all_subs = [c for subs in pillars.values() for c in subs]
missing = [c for c in all_subs if c not in raw.columns]

if missing:
    print('Warning: the following subfeatures are missing from the CSV and will be ignored:', missing)

In [25]:
# Compute D-CRITIC weights
pillar_weights = {}
pillar_dcor_mats = {}

for p, sublist in pillars.items():
    present = [s for s in sublist if s in raw.columns]

    if len(present) == 0:
        continue

    df_sub = raw[present]

    try:
        w, mat = d_critic_weights(df_sub)
    except Exception as e:
        w = pd.Series(1.0 / len(present), index=present)
        mat = pd.DataFrame(np.eye(len(present)), index=present, columns=present)
        
    pillar_weights[p] = w
    pillar_dcor_mats[p] = mat

# Display weights
for p, w in pillar_weights.items():
    print('\nPillar:', p)
    print(w)



Pillar: transportation
Public Transport    0.024360
CO2                 0.957046
Country.1           0.018594
Name: D-CRITIC_weight, dtype: float64

Pillar: air_quality
PM    1.0
dtype: float64

Pillar: waste
Landfill    0.014972
Recycle     0.985028
Name: D-CRITIC_weight, dtype: float64

Pillar: water
Water Reuse     0.997704
Water Stress    0.002296
Name: D-CRITIC_weight, dtype: float64

Pillar: accommodation
Accommodation    0.99742
Temp_x           0.00129
Temp_y           0.00129
Name: D-CRITIC_weight, dtype: float64

Pillar: energy
Energy Use    1.0
dtype: float64


In [26]:
# Compute pillar representative scores per country 
country_df = raw.copy()

for p, w in pillar_weights.items():
    subs = w.index.tolist()
    z_vals = pd.DataFrame(index=country_df.index)

    for s in subs:
        col = pd.to_numeric(country_df[s], errors='coerce')

        if col.dropna().shape[0] < 2 or np.nanstd(col.dropna(), ddof=1) == 0:
            z = pd.Series(0.0, index=col.index)
        else:
            z = (col - col.mean()) / col.std(ddof=1)

        z_vals[s] = np.tanh(z)

    contrib = pd.Series(0.0, index=country_df.index)

    for s in subs:
        st = status.get(s, 'ad')
        sign = 1.0 if st == 'ad' else -1.0
        weight_s = float(w.get(s, 0.0))
        contrib = contrib + sign * weight_s * z_vals[s].fillna(0.0)

    country_df[p + '_score'] = contrib

score_cols = [c for c in country_df.columns if c.endswith('_score')]

print('\nPillar scores (head):')
print(country_df[ ['Country'] + score_cols ].head())

# TOPSIS on pillar-level scores
pillars_present = [p for p in pillars.keys() if p + '_score' in country_df.columns]
ahp_weights = {p: 1.0 for p in pillars_present}  # equal importance placeholder

# Example impacts: assume higher pillar_score is better for all except where domain knowledge says otherwise
impacts = ['+' for _ in pillars_present]
weights_list = [ahp_weights[p] for p in pillars_present]


Pillar scores (head):
       Country  transportation_score  air_quality_score  waste_score  \
0  Afghanistan              0.680926          -0.994777     0.010226   
1       Africa              0.595430           0.000000     0.000000   
2      Albania              0.408846           0.365215    -0.003660   
3      Algeria              0.082211          -0.081922    -0.548149   
4       Angola              0.589522          -0.386400     0.010004   

   water_score  accommodation_score  energy_score  
0     0.549370            -0.609061      0.339032  
1     0.000000            -0.478302      0.207165  
2     0.087808            -0.568545      0.327088  
3    -0.480769            -0.082899     -0.717196  
4    -0.621015            -0.513601     -0.085626  


In [27]:
# prepare df for TOPSIS: ensure Country column exists and unique identifier
if 'Country' not in country_df.columns:
    country_df['Country'] = country_df.index.astype(str)

topsis_input = country_df.copy()

In [28]:
# Run TOPSIS
result = weighted_topsis(topsis_input.reset_index(drop=True), [p + '_score' for p in pillars_present], weights_list, impacts)

print('\nTOPSIS results (top 10):')
print(result[['Country', 'topsis_score', 'rank']].head(10))


TOPSIS results (top 10):
           Country  topsis_score  rank
58         Estonia      0.719523     1
162      Singapore      0.707016     2
192  United States      0.666919     3
69         Germany      0.659453     4
17         Belgium      0.656608     5
127    Netherlands      0.656208     6
88           Italy      0.651603     7
26        Bulgaria      0.649801     8
155          Samoa      0.630852     9
196        Vanuatu      0.630285    10


# Sensitivity Test

In [29]:
# SOBOL Test
def sobol_test_on_pillars(df, pillar_names, impacts, n_samples=512):
    k = len(pillar_names)
    problem = {
        'num_vars': k,
        'names': pillar_names,
        'bounds': [[0, 1]] * k
    }
    sample_matrix = saltelli.sample(problem, n_samples, calc_second_order=False)
    sample_matrix = sample_matrix / sample_matrix.sum(axis=1, keepdims=True)
    winners = []
    scores = []

    for weights in sample_matrix:
        tmp = weighted_topsis(df.copy(), [p + '_score' for p in pillar_names], weights, impacts)
        winners.append(tmp.iloc[0]['Country'])
        scores.append(tmp.iloc[0]['topsis_score'])

    sobol_res = sobol.analyze(problem, np.array(scores), print_to_console=False)
    winner_freq = pd.Series(winners).value_counts(normalize=True)
    
    return sobol_res, winner_freq

In [ ]:
# Run SOBOL test
try:
    sobol_res, winner_freq = sobol_test_on_pillars(result, pillars_present, impacts, n_samples=1024)
    
    print('\nSobol S1 indices for pillars:')
    print(pd.Series(sobol_res['S1'], index=pillars_present))
    print('\nWinner probabilities:')
    print(winner_freq.head())
except Exception as e:
    print('Sobol analysis failed or was too slow in this environment:', e)

/tmp/ipykernel_41421/970771232.py:9: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5.1 Please use `salib.sample.sobol`
  sample_matrix = saltelli.sample(problem, n_samples, calc_second_order=False)


# Redundancy Test

In [ ]:
# Redundancy test
def redundancy_test(df, criteria, weights, impacts):
    baseline = weighted_topsis(df.copy(), criteria, weights, impacts)
    baseline_scores = baseline['topsis_score'].values
    baseline_ranks = baseline['rank'].values
    results = []
    for i, removed in enumerate(criteria):
        reduced_criteria = [c for c in criteria if c != removed]
        reduced_impacts = [impacts[j] for j, c in enumerate(criteria) if c != removed]
        reduced_weights = [weights[j] for j, c in enumerate(criteria) if c != removed]
        reduced_weights = np.array(reduced_weights)
        reduced_weights = reduced_weights / reduced_weights.sum()
        reduced = weighted_topsis(df.copy(), reduced_criteria, reduced_weights, reduced_impacts)
        reduced = reduced.sort_values('Country')
        base_sorted = baseline.sort_values('Country')
        corr, _ = spearmanr(base_sorted['topsis_score'], reduced['topsis_score'])
        rank_shift = np.abs(base_sorted['rank'] - reduced['rank'])
        results.append({
            'removed_criterion': removed,
            'correlation': corr,
            'avg_rank_shift': rank_shift.mean(),
            'max_rank_shift': rank_shift.max()
        })
    return pd.DataFrame(results).sort_values('correlation')



In [ ]:
# Run redundancy test
try:
    redundancy_results = redundancy_test(result, [p + '_score' for p in pillars_present], weights_list, impacts)
    
    print('\nRedundancy test:')
    print(redundancy_results)
except Exception as e:
    print('Redundancy test failed:', e)

out_dir = 'critic_weight'

os.makedirs(out_dir, exist_ok=True)

for p, w in pillar_weights.items():
    w.to_csv(os.path.join(out_dir, f'd_weight_{p}.csv'))

print('\nSaved D-CRITIC pillar weights to critic_weight/ folder.')


Redundancy test:
      removed_criterion  correlation  avg_rank_shift  max_rank_shift
2           waste_score     0.802115       26.147783             104
3           water_score     0.823535       24.068966             122
5          energy_score     0.897559       17.221675             100
1     air_quality_score     0.904705       20.689655              60
4   accommodation_score     0.916625       19.201970              70
0  transportation_score     0.925929       18.039409              71

Saved D-CRITIC pillar weights to critic_weight/ folder.
